In [1]:
import scipy.stats
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_pickle("accidents.pkl.gz")

# Hypotéza 1
#### Na silnicích první třídy se při nehodách umíralo se stejnou pravděpodobností jako dálnicích.
Z původního dataframu jsou vyfiltrovány nehody, které se staly na silnicích 1. třídy a na dálnicích. Jsou vybrány sloupce s typem silnice a počtem úmrtí v nehodě
Je vytvořen nový dataframe s 1. sloupcem určujícím typ silnice a 2. sloupcem, který určuje, zda se jednalo o fatální nehodu - tedy počet úmrtí byl alespoň 1.
Poté je vytvořena kontingenční tabulka, která určuje počet fatálních a nefatálních nehod na jednotlivých typech silnic. 

In [3]:
df_filtered = df.loc[df["p36"].isin([0,1]),["p13a", "p36"]]
df_hyp1 = pd.DataFrame({
    'road_type': df_filtered['p36'],
    'fatal': df_filtered['p13a']>0
}).dropna()
ct = pd.crosstab(df_hyp1["road_type"], df_hyp1["fatal"])
print(ct)

fatal      False  True
road_type             
0          24293   166
1          78618   911


Pomocí chí-kvadrát testu je určena korelace a očekávané hodnoty za předpokladu platnosti hypotézy H0 - tedy neexistující korelace mezi typem silnice a počtem fatálních nehod

In [4]:
chi,p,degrees,expected = scipy.stats.chi2_contingency(ct)
print('P:', p)
print('Expected:', expected)

Chi: 39.314921530329016
P: 3.6067450279444316e-10
Expected: [[24205.67901104   253.32098896]
 [78705.32098896   823.67901104]]


Hodnota korelačního koeficientu je menší než 0.05, korelace mezi typem silnice (pouze 1. třídy a dálnice) a fatálními nehodami tedy existuje se spolehlivostí 95%. Nehody na silnicích 1. třídy byly fatální častěji, než je očekáváno (dle hodnot expected a reálných hodnot)

# Hypotéza 2
#### Při nehodách vozidel značky Škoda je škoda na vozidle nižší než při nehodách vozidel Audi.


In [5]:
df_skoda = df[["p45a", "p53"]].dropna()
skoda = df_skoda.loc[df_skoda["p45a"]==39]
audi = df_skoda.loc[df_skoda["p45a"]==2]
result = scipy.stats.ttest_ind(skoda["p53"],audi["p53"])
print(result)

Ttest_indResult(statistic=-39.12749433594028, pvalue=0.0)


Při nehodách je škoda na vozidlech značky Škoda nižší než na vozidlech značky Audi (hodnota T je záporná). Hodnota p je tak malá, že je interpretována jako 0.0, je tedy na požadované hladině významnosti 0.05